In [1]:
import pandas as pd
import random
import authorization # this is the script we created earlier
import numpy as np
from numpy.linalg import norm

In [2]:
# Считываем данные 12 000 треков из фрейма:
df = pd.read_csv("data.csv")

In [3]:
# Комбинируем столбцы «валентности» и «возбуждения» и создаём для каждого трека единый вектор mood_vec:
df["mood_vec"] = df[["valence", "energy"]].values.tolist()

In [4]:
# Последний этап перед реализацией алгоритма рекомендаций — авторизация для доступа к API Spotify:
sp = authorization.authorize()

In [5]:
def recommend(track_id, ref_df, sp, n_recs = 5):
    
    # Crawl valence and arousal of given track from spotify api
    track_features = sp.track_audio_features(track_id)
    track_moodvec = np.array([track_features.valence, track_features.energy])
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
    # If the input track is in the reference set, it will have a distance of 0, but should not be recommendet
    ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    
    # Return n recommendations
    return ref_df_sorted.iloc[:n_recs]

In [6]:
hyperpop = "3IR3fpipl8fH8TnQrjdyl5?si=8656324593864f1c"
recommend(track_id=hyperpop, ref_df=df, sp=sp, n_recs=5)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,mood_vec,distances
112224,0.585,1956,0.8970,['Sister Rosetta Tharpe'],0.542,152065,0.617,0,2mp2GHI6WwVhiEuvmnJ13t,0.000000,...,0.1670,-5.048,1,When They Ring The Golden Bell,6,1956-03-23,0.0351,111.148,"[0.585, 0.617]",0.001414
1851,0.585,1931,0.7650,['Charlie Chaplin'],0.489,255765,0.615,0,0DIOI3RPy6UGENHZSjHqAd,0.892000,...,0.2110,-5.306,1,Overture / Unveiling the Monument,0,1931,0.1180,140.121,"[0.585, 0.615]",0.001414
1866,0.585,1931,0.7650,['Charlie Chaplin'],0.489,255765,0.615,0,0NIzqSNAePsEtFczBTs7Pe,0.892000,...,0.2110,-5.306,1,Overture / Unveiling the Monument,4,1931,0.1180,140.121,"[0.585, 0.615]",0.001414
147287,0.588,1978,0.0743,['DEVO'],0.573,224093,0.616,0,6sx36Cz7YmbzZzzu1MeRfl,0.043700,...,0.0996,-7.806,0,Mongoloid - 2009 Remaster,23,1978,0.0445,148.148,"[0.588, 0.616]",0.002000
92789,0.587,1930,0.8470,['Giorgos Papasideris'],0.348,192733,0.618,0,3bQr9DH74IsCHxRXuIK38g,0.000518,...,0.1660,-4.499,1,Pethane O Vlahos,0,1930-01-01,0.0393,115.627,"[0.5870000000000001, 0.618]",0.002236
